# Phase 2 - Transform - Model

So we have a SQLite database, and we could've decided that this would be our data lake (a *relational database*). But we've decided to go with the *document database* way, and already put up the work to inspect the tables, find relationships between them and build the document representation.

However, that's all still on paper, so now it's time to actually build it.

We could do it in python using *pandas*, but this is one of those times where knowing SQL will give you a great advantage over others. It's easier, faster (especially if you're dealing with large volumes of data) and much less error prone. To store and manipulate related tabular data is why databases exist, even modern cloud services like Azure (from Microsoft), AWS (Amazon Web Services) or GCP (Google Cloud Platform) support relational databases to process data at huge scales.

**So first, let's talk about SQL (Structured Query Language) and then we'll return back here.**

## Incrementally joining the tables

We'll use the work done in the slides as a guide. Here's what we've come up with:

![Alt text](assets/2-db_diagram.png)

And the final document (or row) we want to have:

![Alt text](assets/2-doc_representation.png)

We don't have to join everything together in a single script, in can be an incremental process (and it will be faster, since at each query the database has less relationships to handle).

### Joining `customer`, `staff` and `rental` to `payment`

It's a simple SQL query:

````sql
select a.*, 
b.name, b.address, b.phone, b."zip code", b.city, b.country, 
c.name, c.address, c.phone, c.city, c.country, c.SID,
d.rental_date, d.return_date, d.inventory_id
from payment a
join customer b on b.ID  = a.customer_id 
join staff c on c.ID = a.staff_id 
join rental d on d.rental_id = a.rental_id;
````

Oops, we have an error: `Result: no such column: d.rental_date`

How is that possible, if the column exist in the rental table?!

### Welcome to the world of freak occurences

We didn't notice, but when we extracted and loaded the data in the previous phase, the columns `rental_date` and `return_date` went with a space at the end.

Actually there's an easy fix for that:

````sql
create table rentals (rental_id bigint, rental_date text, inventory_id bigint, customer_id bigint, return_date text, staff_id bigint);
insert into rentals(rental_id, rental_date, inventory_id, customer_id, return_date, staff_id) select * from rental;
drop table rental;
````

Ok, so now we have a new table called `rentals` with all the data from the `rental` table. Let's rewrite and and try the query again:

````sql
select a.*, 
b.name, b.address, b.phone, b."zip code", b.city, b.country, 
c.name, c.address, c.phone, c.city, c.country, c.SID,
d.rental_date, d.return_date, d.inventory_id
from payment a
join customer b on b.ID  = a.customer_id 
join staff c on c.ID = a.staff_id 
join rentals d on d.rental_id = a.rental_id;
````

That did it. Now we should rename the columns from the `staff` table because they're equal to the `customer`table. While we're at it, let's get rid of any spaces in our column names and rename 'zip code' to `zip_code`. Let's rewrite it once again:

````sql
select a.*, 
b.name, b.address, b.phone, b."zip code" as zip_code, b.city, b.country, 
c.name as staff_name, c.address as staff_address, c.phone as staff_phone, c.city as staff_city, c.country as staff_country, c.SID,
d.rental_date, d.return_date, d.inventory_id
from payment a
join customer b on b.ID  = a.customer_id 
join staff c on c.ID = a.staff_id 
join rentals d on d.rental_id = a.rental_id;
````

Notice the result of the query: `Result: 16044 rows`. It tells us that it went well, because the `payment`table also has 16044 rows.

Also notice that we've kept the `SID` and the `inventory_id` fields from the `staff`and `rentals` table, because we'll need them in the following joins.

### Joining `store` and `inventory` to our query

Here it is:

````sql
select a.*, 
b.name, b.address, b.phone, b."zip code" as zip_code, b.city, b.country, 
c.name as staff_name, c.address as staff_address, c.phone as staff_phone, c.city as staff_city, c.country as staff_country, c.SID,
d.rental_date, d.return_date, d.inventory_id,
e.address as store_address, e.district as store_district, e.city as store_city,
f.film_id 
from payment a
join customer b on b.ID  = a.customer_id 
join staff c on c.ID = a.staff_id 
join rentals d on d.rental_id = a.rental_id
join store e on e.store_id = c.SID
join inventory f on f.inventory_id = d.inventory_id;
````

Did you noticed how the query took a lot more time compared to the previous one? `Result: 16044 rows returned in 35032ms`.

That happens because our tables don't have any indexes or primary keys (a different form of indexing), so we're forcing SQLite to sweep the entire content of the tables searching for the relationships. Let's help him out by runnning these queries:

````sql
-- add primary keys and create indexes
-- customer table
alter table customer rename to old_customer;
create table "customer"(
  "id" text,
  "name" text,
  "address" text,
  "zip code" text,
  "phone" text,
  "city" text,
  "country" text,
  "notes" text,
  constraint customer_pk primary key (id)
);
insert into customer select * from old_customer;
drop table old_customer;

-- staff table
alter table staff rename to old_staff;
create table staff (
	"id" bigint, 
	name text, 
	address text, 
	phone bigint, 
	city text, 
	country text, 
	"sid" bigint,
	constraint staff_pk primary key (id)
);
insert into staff select * from old_staff;
drop table old_staff;

-- rentals table
alter table rentals rename to old_rentals;
create table rentals ( 
	rental_id bigint, 
	rental_date text, 
	inventory_id bigint, 
	customer_id bigint, 
	return_date text, 
	staff_id bigint,
	constraint rentals_pk primary key (rental_id)
);
insert into rentals select * from old_rentals;
drop table old_rentals;

-- store table
alter table store rename to old_store;
create table store (
	store_id bigint, 
	address text, 
	district text, 
	city text,
	constraint store_pk primary key (store_id)
);
insert into store select * from old_store;
drop table old_store;

-- inventory table
alter table inventory rename to old_inventory;
create table "inventory"(
	"inventory_id" text,
	"film_id" text,
	constraint inventory_pk primary key (inventory_id)
);
insert into inventory select * from old_inventory;
drop table old_inventory;

-- index creation
create index "id_customers" on "payment" ("customer_id" asc);
create index "id_staff" on "payment" ("staff_id" asc);
create index "id_rental" on "payment" ("rental_id" asc);
create index "id_store" on "staff" ("sid" asc);
create index "id_inventory" on "rentals" ("inventory_id" asc);

-- reindex
reindex payment;
reindex staff;
reindex rentals;
````

Let's try to run the same query again:

````sql
select a.*, 
b.name, b.address, b.phone, b."zip code" as zip_code, b.city, b.country, 
c.name as staff_name, c.address as staff_address, c.phone as staff_phone, c.city as staff_city, c.country as staff_country, c.SID,
d.rental_date, d.return_date, d.inventory_id,
e.address as store_address, e.district as store_district, e.city as store_city,
f.film_id 
from payment a
join customer b on b.ID  = a.customer_id 
join staff c on c.ID = a.staff_id 
join rentals d on d.rental_id = a.rental_id
join store e on e.store_id = c.SID
join inventory f on f.inventory_id = d.inventory_id;
````

Awesome: `Result: 16044 rows returned in 5173ms`

Since we'll have to do this for the other tables as well, let's take the time to add primary keys and indexes to them. Run the queries below:

````sql
-- film table
alter table film rename to old_film;
CREATE TABLE film (
	film_id INTEGER,
	title VARCHAR(27),
	description VARCHAR(130),
	release_year INTEGER,
	language_id INTEGER,
	original_language_id VARCHAR(1),
	rental_duration INTEGER,
	rental_rate REAL,
	"length" INTEGER,
	replacement_cost REAL,
	rating VARCHAR(5),
	special_features VARCHAR(54),
	constraint film_pk primary key (film_id)
);
insert into film select * from old_film;
drop table old_film;

-- language table
alter table language rename to old_language;
CREATE TABLE "language" (
	language_id INTEGER,
	name VARCHAR(8),
	constraint language_pk primary key (language_id)
);
insert into language select * from old_language;
drop table old_language;

-- category table
alter table category rename to old_category;
CREATE TABLE "category" (
	category_id INTEGER,
	name VARCHAR(11),
	constraint category_pk primary key (category_id)
);
insert into category select * from old_category;
drop table old_category;

-- actor table
alter table actor rename to old_actor;
CREATE TABLE actor (
	actor_id INTEGER,
	first_name VARCHAR(11),
	last_name VARCHAR(12),
	constraint actor_pk primary key (actor_id)
);
insert into actor select * from old_actor;
drop table old_actor;

-- index creation
create index "id_film" on "inventory" ("film_id" asc);
create index "id_language" on "film" ("language_id" asc);
create index "id_film_category" on "film_category" ("film_id" asc);
create index "id_category" on "film_category" ("category_id" asc);
create index "id_film_actor" on "film_actor" ("film_id" asc);
create index "id_actor" on "film_actor" ("actor_id" asc);

-- reindex
reindex inventory;
reindex film;
reindex film_category;
reindex film_actor;
````

### Joining the original film db tables: `film`, `language`, `category` and `actor`

Let's leave our previous joins alone for now and focus on joining all these tables together.

Are you up for the task?

Here it is:

````sql
select a.film_id, a.title, a.description, a.release_year, a.rental_duration, a.rental_rate, a.length, a.replacement_cost, a.rating, a.special_features,
b.name as language, 
d.name as category, 
f.first_name || ' ' || f.last_name as actor -- this is how concat is made in SQLite, for other db's use concat()
from film a
join language b on b.language_id = a.language_id
join film_category c on c.film_id = a.film_id
join category d on d.category_id = c.category_id
join film_actor e on e.film_id = a.film_id
join actor f on f.actor_id = e.actor_id;
````

Look at the result message: `Result: 5462 rows`. It should have given us the same number as the `film` table, which is 1000. Can you explain it?

So there's nothing wrong with our query, but since each movie has more than one actor it will create duplicate rows where only the actor name is different:

![Alt text](assets/2-denormalization.png)

A decision has to be made at this point. Does it stay like this or, for example, should we put all the actor's names together in one cell?

![Alt text](assets/2-denormalization_2.png)

There are pro's and con's for each option and it will have it's impact along the road, so the most important question is: **will the majority of my analysis be focused on the actors of a film?**

In my opinion, that's a definitive no, so we're better off with putting all the actors in the same cell.

So here's our rewritten query:

````sql
select a.film_id, a.title, a.description, a.release_year, a.rental_duration, a.rental_rate, a.length, a.replacement_cost, a.rating, a.special_features,
b.name as language, 
d.name as category, 
GROUP_CONCAT(f.first_name || ' ' || f.last_name) as actor -- we used group_concat()
from film a
join language b on b.language_id = a.language_id
join film_category c on c.film_id = a.film_id
join category d on d.category_id = c.category_id
left join film_actor e on e.film_id = a.film_id -- we swithed to left join because not all films have actors
left join actor f on f.actor_id = e.actor_id -- we swithed to left join because not all films have actors
group by a.film_id; -- to group_concat we have to group the records by something
````

Now we have the same result: `Result: 1000 rows`

### Joining the two queries

Ok, so now we have one query that joins some tables and another that joins the others, it's time to join everything together.

We could write a new query with all the joins, but that's not a very clever idea, especially if we were dealing with a high volume of data.

Here's another idea: we'll send the result of the queries into a view, so that we don't have to keep doing the join everytime we want to query it.

````sql
create view joined_data as
    select first.*, second.*
    from
        (select a.*, 
        b.name, b.address, b.phone, b."zip code" as zip_code, b.city, b.country, 
        c.name as staff_name, c.address as staff_address, c.phone as staff_phone, c.city as staff_city, c.country as staff_country, -- c.sid,
        d.rental_date, d.return_date, -- d.inventory_id,
        e.address as store_address, e.district as store_district, e.city as store_city,
        f.film_id 
        from payment a
        join customer b on b.id  = a.customer_id 
        join staff c on c.id = a.staff_id 
        join rentals d on d.rental_id = a.rental_id
        join store e on e.store_id = c.sid
        join inventory f on f.inventory_id = d.inventory_id) as first
    join
        (select a.film_id, a.title, a.description, a.release_year, a.rental_duration, a.rental_rate, a.length, a.replacement_cost, a.rating, a.special_features,
        b.name as language, 
        d.name as category, 
        group_concat(f.first_name || ' ' || f.last_name) as actor
        from film a
        join language b on b.language_id = a.language_id
        join film_category c on c.film_id = a.film_id
        join category d on d.category_id = c.category_id
        left join film_actor e on e.film_id = a.film_id
        left join actor f on f.actor_id = e.actor_id
        group by a.film_id) as second 
    on second.film_id = first.film_id;
````

Now you can query it:

````sql
select * from joined_data;
````

---

Just in case you're wondering what would happen if we've joined everything together:

````sql
select a.*, 
b.name, b.address, b.phone, b."zip code" as zip_code, b.city, b.country, 
c.name as staff_name, c.address as staff_address, c.phone as staff_phone, c.city as staff_city, c.country as staff_country,
d.rental_date, d.return_date,
e.address as store_address, e.district as store_district, e.city as store_city,
f.film_id,
g.film_id, g.title, g.description, g.release_year, g.rental_duration, g.rental_rate, g.length, g.replacement_cost, g.rating, g.special_features,
h.name as language, 
j.name as category, 
group_concat(l.first_name || ' ' || l.last_name) as actor 
from payment a
join customer b on b.id  = a.customer_id 
join staff c on c.id = a.staff_id 
join rentals d on d.rental_id = a.rental_id
join store e on e.store_id = c.sid
join inventory f on f.inventory_id = d.inventory_id
join film g on g.film_id = f.film_id 
join language h on h.language_id = g.language_id
join film_category i on i.film_id = g.film_id
join category j on j.category_id = i.category_id
left join film_actor k on k.film_id = g.film_id
left join actor l on l.actor_id = k.actor_id
group by g.film_id;
````

You can try to run this on your own free time: I gave up after 20min.

### Why a view and not a table?

This is an important point. 

Although it's not the case here, imagine that your tables would be regularly updated with new data; our two big joins will read that new data, thus ensuring that we won't miss a thing. 

A view has no content, unlike a table; a view will run some SQL everytime we query it. Well we've just told our view to run those joins, didn't we? So this means that everytime we do a select or import the view into *pandas*, we'll always get the most updated data available!

## That's a wrap!

Ok, so we finally have a unified data model. In the process we've already performed some data transformations:

* removed unnecessary columns
* concatenated columns
* renamed columns for better readability

Now it's time to see what we can do regarding the content.

Open *2.2-transform-data.ipynb* and let's move forward.